# Yapay Sinir Ağları ile Meme Kanserinin Teşhisi

Dünya Kanser Araştırma Fonu'na göre Meme Kanseri kadınlarda en sık görülen kanser türü. 2018 yılında Meme Kanseri vaka sayısı 2.088.849 ve % 25.4'üne kanser teşhisi konuldu.

Bu rakamlar erken aşamada Meme Kanseri tespit etmek için bir çözüm üzerinde çalışmaya ikna edecek kadar dikkat çekmektedir.

* Aşağıdaki siteden 699 Meme kanseri şüphelisi hastanın verisini .data formatında bulabilirsiniz. Sizinle paylaşacağım github sayfamda verinin .csv formatı mevcut olacak.

https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

Verimizi data adıyla pandas kütüphanesi kullanarak kaydedelim

In [2]:
import pandas as pd
from google.colab import files
file = files.upload()
data = pd.read_csv("breastCancer.csv", header = None)

Saving breastCancer.csv to breastCancer.csv


In [16]:
data.head(15)

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0
5,1017122,8,10,10,8,7,10,9,7,1,1
6,1018099,1,1,1,1,2,10,3,1,1,0
7,1018561,2,1,2,1,2,1,3,1,1,0
8,1033078,2,1,1,1,2,1,1,1,5,0
9,1033078,4,2,1,1,2,1,2,1,1,0


Verimize ait sayı-tür bilgileri

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
0     699 non-null int64
1     699 non-null int64
2     699 non-null int64
3     699 non-null int64
4     699 non-null int64
5     699 non-null int64
6     699 non-null int64
7     699 non-null int64
8     699 non-null int64
9     699 non-null int64
10    699 non-null int64
dtypes: int64(11)
memory usage: 60.1 KB


6. indexte bazı veriler ? olarak girilmiş önce bunları 0 olarak değiştirelim daha sonra kolonun ortalamasını alarak bu 0 yazdığımız hücrelere ortalamasını yazalım

In [23]:
df = pd.DataFrame(data)
df[6] = df[6].replace('?',0)
df[6] = df[6].astype(int)

mean = int(df[6].mean())
df[6] = df[6].replace(0,mean)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


6.indexte hangi değerden kaç tane var görelim

In [22]:
df = pd.DataFrame(data)
df[6].value_counts()

1     402
10    132
3      44
5      30
2      30
8      21
4      19
9       9
7       8
6       4
Name: 6, dtype: int64

Veri setimiz 699 hastaya ait tümöre ait bilgilerini içeriyor.
İlk Kolon hastalara ait ID bilgisi

Diğer kolonlardan 1-9 arası tüm kolonlar tümöre ait değerler. Bu değerler 1-10 arası. 1'e ne kadar yakınsa iyi huylu, 10 a ne kadar yakınsa kötü huylu demek. 10 nolu kolon ise hastanın tümörünün sonuç olarak iyi huylu mu (2 değeri) yoksa kötü huylu mu(4 değeri) olduğunun bilgisini tutuyor. Biz 1-9 kolonlarındaki değerlere bakarak 10 nolu kolondaki sonuç bilgisini yapay sinir ağı ile tahmin etmeye çalışacağız.

10 nolu kolonda 2 olan değerleri 0, 4 olan değerleri 1 olarak değiştirelim

In [24]:
df[10] = df[10].replace(2,0).replace(4,1)
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0
5,1017122,8,10,10,8,7,10,9,7,1,1
6,1018099,1,1,1,1,2,10,3,1,1,0
7,1018561,2,1,2,1,2,1,3,1,1,0
8,1033078,2,1,1,1,2,1,1,1,5,0
9,1033078,4,2,1,1,2,1,2,1,1,0


Verimizi hazırladığımıza göre artık yapay sinir ağımızı oluşturabiliriz.Önce X değişkeni olarak 1-9 arası kolonlar Y değişkeni olarak 10. kolonu seçelim Verimizi %20 test %80 train olarak bölelim

In [0]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, 1:10]
Y = df.iloc[:, 10]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

X değişkenlerimizi normalize edelim

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


Keras ile sinir ağımızı oluşturalım

In [8]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [0]:
classifier = Sequential()

input layer ve hidden layerlerimizi oluşturalım. çIKIŞ 1-0 değerleri istediğimizden aktivasyon fonksiyonumuz sigmoid olacak

In [10]:
classifier.add(Dense(units = 5, activation = 'relu', input_dim=9))
classifier.add(Dense(units = 3, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


Yapay Sinir Ağımızı adam optimizer, mean_squared_error ve accuracy metriği ile compile edelim

In [0]:
classifier.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

100 epoch ve 10 batch_size ile eğitimimize başlayalım

In [12]:
classifier.fit(X_train, Y_train, batch_size = 10, epochs = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
559/559 [==============================] - 3s 5ms/step - loss: 0.2763 - acc: 0.6673
Epoch 2/100
559/559 [==============================] - 0s 292us/step - loss: 0.1905 - acc: 0.7639
Epoch 3/100
559/559 [==============================] - 0s 294us/step - loss: 0.1274 - acc: 0.8265
Epoch 4/100
559/559 [==============================] - 0s 298us/step - loss: 0.0876 - acc: 0.8927
Epoch 5/100
559/559 [==============================] - 0s 274us/step - loss: 0.0653 - acc: 0.9284
Epoch 6/100
559/559 [==============================] - 0s 289us/step - loss: 0.0531 - acc: 0.9356
Epoch 7/100
559/559 [==============================] - 0s 289us/step - loss: 0.0451 - acc: 0.9463
Epoch 8/100
559/559 [==============================] - 0s 277us/step - loss: 0.0394 - acc: 0.9499
Epoch 9/100
559/559 [==============================] - 0s 275us/step - loss: 0.0350 - acc: 0.9571
Epoch 10/100
559/559 [==============================] - 0s 308us/step - 

Test olarak ayırdığımız X değerlerini tahmin edelim

In [13]:
Y_pred = classifier.predict(X_test)
Y_pred = [ 1 if y>=0.5 else 0 for y in Y_pred]
print(Y_pred)

[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0]


Confussion matrix ile baktığımızda 0 ile encode ettiğimiz 85 değerin 3 ünü yanlış bilmiş 82 sini doğru bilmiş

55 kötü huylu tümörün sadece birini yanlış bilmiş

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)
print(cm)

[[82  3]
 [ 1 54]]


# Faydalı olması dileğiyle